## doc2vec training excercise

In this excercise, you will train a Paragraph Vectors / doc2vec model using gensim. You can find information on the gensim doc2vec api here: https://radimrehurek.com/gensim/models/doc2vec.html

N.B. You should be using Python 3 for this.

The data folder contains a train and test set with small sets of documents from the "20 newsgroups" dataset.

What we're going to do is the following:
* Read a dataset with documents
* Transform each document into a list of tokens (words)
* Train a doc2vec model (DM)
* Train a second model (DBOW)
* Inspect the outcomes a bit

In [1]:
import os
from gensim.models import doc2vec
from gensim.utils import simple_preprocess

In [2]:
# generic settings
HOMEDIR = './'
CORPUS_FILE = os.path.join(HOMEDIR, "data/train_docs.txt")

# file names for the models we'll be creating
MODEL_FILE_DM = os.path.join(HOMEDIR, "models/doc2vec_DM_v20171229.bin")
MODEL_FILE_DBOW = os.path.join(HOMEDIR, "models/doc2vec_DBOW_v20171229.bin")

**Read the corpus. Each line is a document / paragraph. Optionally preprocess it first.**

In [3]:
flg_preprocess = False

if flg_preprocess:
    # quick & simple approach
    docs = doc2vec.TaggedLineDocument(CORPUS_FILE)
else:
    # with pre-processing
    with open(CORPUS_FILE, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        docs = [simple_preprocess(line, deacc=False, min_len=1) for line in lines]
        docs = [doc2vec.TaggedDocument(doc, tags=[i]) for i, doc in enumerate(docs)]

In [4]:
# have a look at the data
docs[0]

TaggedDocument(words=['anarchism', 'is', 'a', 'political', 'philosophy', 'that', 'advocates', 'self', 'governed', 'societies', 'with', 'voluntary', 'institutions', 'these', 'are', 'often', 'described', 'as', 'stateless', 'societies', 'but', 'several', 'authors', 'have', 'defined', 'them', 'more', 'specifically', 'as', 'institutions', 'based', 'on', 'non', 'hierarchical', 'free', 'associations', 'anarchism', 'holds', 'the', 'state', 'to', 'be', 'undesirable', 'unnecessary', 'or', 'harmful', 'while', 'anti', 'statism', 'is', 'central', 'anarchism', 'entails', 'opposing', 'authority', 'or', 'hierarchical', 'organisation', 'in', 'the', 'conduct', 'of', 'human', 'relations', 'including', 'but', 'not', 'limited', 'to', 'the', 'state', 'system'], tags=[0])

## Training a DM (Distributed Memory) model

In [8]:
# train DM model
model_dm = doc2vec.Doc2Vec(docs, 
                           vector_size=200, # vector size, should be the same size as pre-trained embedding size when not using dm_concat
                           window=10, # window size for word context, on each side
                           min_count=1, # minimum nr. of occurrences of a word
                           sample=1e-5, # threshold for undersampling high-frequency words
                           workers=4, # for multicore processing
                           hs=0, # if 1, use hierarchical softmax; if 0, use negative sampling
                           dm=1, # if 1 use PV-DM, if 0 use PM-DBOW
                           negative=5, # how many words to use for negative sampling
                           dbow_words=1, # train word vectors
                           dm_concat=1, # concatenate vectors or sum/average them?
                           epochs=100 # nr of epochs to train
                          )

In [9]:
# save it for later use
model_dm.save(MODEL_FILE_DM)

## Training a DBOW (Distributed Bag Of Words) model

**_Excercise 1: Train a DBOW model_**

It's very similar to the previous command. What should you change?

In [10]:
# train DBOW model
# ...enter your code here...
model_dbow = doc2vec.Doc2Vec(docs, 
                            vector_size=200, # vector size, should be the same size as pre-trained embedding size when not using dm_concat
                            window=10, # window size for word context, on each side
                            min_count=1, # minimum nr. of occurrences of a word
                            sample=1e-5, # threshold for undersampling high-frequency words
                            workers=4, # for multicore processing
                            hs=0, # if 1, use hierarchical softmax; if 0, use negative sampling
                            dm=0, # if 1 use PV-DM, if 0 use PM-DBOW
                            negative=5, # how many words to use for negative sampling
                            dbow_words=1, # train word vectors
                            epochs=100 # nr of epochs to train
                            )

In [8]:
# solution
model_dbow = doc2vec.Doc2Vec(docs, 
                            size=200, # vector size, should be the same size as pre-trained embedding size when not using dm_concat
                            window=10, # window size for word context, on each side
                            min_count=1, # minimum nr. of occurrences of a word
                            sample=1e-5, # threshold for undersampling high-frequency words
                            workers=4, # for multicore processing
                            hs=0, # if 1, use hierarchical softmax; if 0, use negative sampling
                            dm=0, # if 1 use PV-DM, if 0 use PM-DBOW
                            negative=5, # how many words to use for negative sampling
                            dbow_words=1, # train word vectors
                            iter=100 # nr of epochs to train
                            )

========= END OF EXERCISE ============

In [11]:
# also save this one
model_dbow.save(MODEL_FILE_DBOW)

## **Question: Look at the model files that are now created in the models directory. Can you explain why there are 2 files for the DM model, but only 1 for the DBOW model?**

In [12]:
def show_most_similar(model, docs, ref_doc_id):
    """
    For a given document, display the most similar ones in the corpus
    """
    def print_doc(doc_id):
        doc_txt = ' '.join(docs[doc_id].words)
        print("[Doc {}]: {}".format(doc_id, doc_txt))
        
    print("[Original document]")
    print_doc(ref_doc_id)
    print("\n[Most similar documents]")
    for doc_id, similarity in model.docvecs.most_similar(ref_doc_id, topn=3):
        print("-----------------")
        print("similarity: {}".format(similarity))
        print_doc(doc_id)


In [13]:
show_most_similar(model_dbow, list(docs), 200)

[Original document]
[Doc 200]: single scattering albedo is used to define scattering of electromagnetic waves on small particles it depends on properties of the material lrb refractive index rrb the size of the particle or particles and the wavelength of the incoming radiation

[Most similar documents]


C:\Users\uSER\AppData\Local\Temp\ipykernel_34220\2777213050.py:12: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  for doc_id, similarity in model.docvecs.most_similar(ref_doc_id, topn=3):


-----------------
similarity: 0.818175196647644
[Doc 165]: the overall albedo of the moon is around but it is strongly directional and non lambertian displaying also a strong opposition effect although such reflectance properties are different from those of any terrestrial terrains they are typical of the regolith surfaces of airless solar system bodies
-----------------
similarity: 0.811142086982727
[Doc 148]: albedo lrb rrb or reflection coefficient derived from latin albedo whiteness lrb or reflected sunlight rrb in turn from albus white is the diffuse reflectivity or reflecting power of a surface
-----------------
similarity: 0.8111039996147156
[Doc 162]: astronomical albedo


## Prediction phase

In [14]:
test_data_file = os.path.join(HOMEDIR, "data/test_docs.txt")

In [15]:
# read test data: each line into a list of tokens
with open(test_data_file, "r") as f:
    test_docs = [ x.strip().split() for x in f.readlines() ]

In [16]:
# inference hyper-parameters
start_alpha=0.01
infer_epoch=1000

Create the embeddings for the test documents. Remember: this is an inference step that actually trains a network.

In [18]:
# test_docvecs = [model_dm.infer_vector(d, alpha=start_alpha, steps=infer_epoch) for d in test_docs]

infer_epoch = 20  # Number of inference epochs

test_docvecs = [model_dm.infer_vector(d, alpha=start_alpha, epochs=infer_epoch) for d in test_docs]


In [19]:
# see what one document embedding looks like
test_docvecs[0]

array([ 2.58632842e-03,  3.63510568e-03, -1.21472322e-03, -1.15607830e-03,
        7.26221242e-06,  1.88632961e-03, -1.15984026e-03,  2.83681066e-03,
        2.13623280e-03, -1.63606170e-03,  6.09889627e-04, -9.48709494e-05,
       -2.53189425e-03,  2.51582661e-03, -1.72592211e-03, -4.17035818e-03,
        7.85952667e-04, -7.88370598e-05,  3.83347913e-04, -1.40118389e-03,
        2.93286459e-04,  3.21178860e-03,  1.74682192e-03, -4.14444925e-03,
       -1.47055055e-03, -2.33964832e-03,  1.89658953e-03, -1.56889553e-03,
        1.53461634e-03, -1.19663282e-05, -5.22329099e-03,  2.03342712e-03,
        2.00471375e-03, -8.12001061e-03,  1.19660384e-04, -3.38673429e-03,
        1.12576061e-03, -5.53413620e-03, -4.18214547e-03, -3.31581896e-03,
        1.56260678e-03, -4.57505463e-03,  3.84328421e-03, -1.67247851e-03,
        1.23591931e-03, -2.89132074e-03, -4.92465962e-03,  1.56075251e-03,
        3.86150088e-03, -1.12858892e-03,  3.01103736e-03,  3.27742528e-05,
        1.61835662e-04, -

### The excercise continues in the next notebook!